In [4]:
import pandas as pd 
import sqlite3
from openpyxl import Workbook
from openpyxl.styles import Border, Side, PatternFill


In [2]:
conn = sqlite3.connect("Cemix_database_test.db")

In [18]:
query = """
            SELECT c.date as 'Date de Shift', c.heure as 'Heure de Shift',
            c.ligne as 'Ligne', c.shift as 'Shift', c.operateur_mix as 'Operateur Mix', 
            c.operateur_ensacheuse  as 'Operateur Ensacheuse',
            c.clarist_m  as 'Clarist M', c.clariste_p as 'Clariste P',
            c.aide_magasinier as 'Aide Magasinier',
            p.date as 'Date de Palette', p.numero_palette as 'Numero de Palette' ,
            a.nom_article as 'Article', p.nombre_de_sac as 'Nombre de Sac',
            p.Commentair as 'Error Commentair', p.poids as 'Poids',
            p.echantillon_10Kg as 'Echantillon 10Kg',
            p.echantillon_4Kg as 'Echantillon 4Kg',
            p.duration_min as 'Palatte_duration',
            p.ecart_by_10 as 'Ecart'
            FROM cemix_info c
            join palette p on p.cemix_main_id = c.id
            join article a on a.id = p.article_id
            where DATE(c.date) = DATE('now', '-2 day') and c.ligne = 'line_1';
        """
df_all_day = pd.read_sql_query(query, conn)

In [19]:
df_all_day

,Date de Shift,Heure de Shift,Ligne,Shift,Operateur Mix,Operateur Ensacheuse,Clarist M,Clariste P,Aide Magasinier,Date de Palette,Numero de Palette,Article,Nombre de Sac,Error Commentair,Poids,Echantillon 10Kg,Echantillon 4Kg,Palatte_duration,Ecart
0,2023-09-21,07:00:00,line_1,Shift_1,Ziad Fellah,Karim Mahmoud,clarist_m_1,clarist_p_1,Oussama,2023-09-21 07:15:20,P001,MARTOUBY MN,10,,100.5,Non,Non,15.33,5.33
1,2023-09-21,07:00:00,line_1,Shift_1,Ziad Fellah,Karim Mahmoud,clarist_m_1,clarist_p_1,Oussama,2023-09-21 07:55:26,P002,MARTOUBY PX,8,,75.2,Oui,Non,40.10,30.10
2,2023-09-21,07:00:00,line_1,Shift_1,Ziad Fellah,Karim Mahmoud,clarist_m_1,clarist_p_1,Oussama,2023-09-21 09:45:33,P003,MARTOUBY MN,10,Défaut mécanique mélangeur,100.5,Non,Oui,110.12,100.12
3,2023-09-21,07:00:00,line_1,Shift_1,Ziad Fellah,Karim Mahmoud,clarist_m_1,clarist_p_1,Oussama,2023-09-21 14:40:45,P004,MARTOUBY PX,8,Défaut électrique mélangeur,75.2,Oui,Non,295.20,285.20
4,2023-09-21,15:00:00,line_1,Shift_2,Ziad Fellah,Karim Mahmoud,clarist_m_1,clarist_p_2,Reda,2023-09-21 15:10:13,P005,MARTOUBY P,10,Manque de sable,100.5,Non,Oui,9.00,-1.00
5,2023-09-21,15:00:00,line_1,Shift_2,Ziad Fellah,Karim Mahmoud,clarist_m_1,clarist_p_2,Reda,2023-09-21 19:40:45,P006,MARTOUBY XTRA FINO,8,,75.2,Oui,Non,270.53,260.53
6,2023-09-21,23:00:00,line_1,Shift_3,Ziad Fellah,Karim Mahmoud,clarist_m_3,clarist_p_1,Oussama,2023-09-21 23:32:16,P007,DECOR PA,8,,75.2,Non,Oui,32.27,22.27
7,2023-09-21,23:00:00,line_1,Shift_3,Ziad Fellah,Karim Mahmoud,clarist_m_3,clarist_p_1,Oussama,2023-09-22 02:17:00,P008,MARTOUBY FN BLANC,10,Défaut Bouche 2 Ensacheuse,100.5,Oui,Oui,164.73,154.73
8,2023-09-21,23:00:00,line_1,Shift_3,Ziad Fellah,Karim Mahmoud,clarist_m_3,clarist_p_1,Oussama,2023-09-22 05:55:30,P009,MARTOUBY MN,8,,75.2,Non,Non,218.50,208.50


In [4]:
def sum_positive(x):
    return x[x > 0].sum()

def sum_Echantt(x):
    return x[x == "Oui"].count()

def sum_error(x):
    return '\n'.join(x[(x != "") | (x != None)])

In [20]:
agg_functions = {
    'Palatte_duration' : 'sum',
    'Numero de Palette': 'count', 
    'Nombre de Sac': 'sum',
    'Ecart': lambda x: x[x > 0].sum(),
    "Error Commentair": lambda x: '___'.join(x[(x != "") | (x != None)]),
    'Echantillon 10Kg': lambda x: x[x == "Oui"].count(),
    'Echantillon 4Kg': lambda x: x[x == "Oui"].count(),
    'Poids' : "sum",
}


In [21]:
dpt_golabl = 10
df_synthes = df_all_day.groupby(["Shift"]).agg(agg_functions).reset_index()

df_synthes.rename(columns = {
                            "Palatte_duration": "Tottal Heure Travaile", 
                            "Numero de Palette": "Nombre De Palette Produite",
                            "Ecart" : "Nombre d'heure d'arret", 
                            "Nombre de Sac": "Nombre Total SAC", 
                            "Poids" : "Tottal Poids", 
                            "Echantillon 10Kg": "Tottal Echantillon 10Kg", 
                            "Echantillon 4Kg": "Tottal Echantillon 4Kg",
                            "Error Commentair": "Cause"
                            }, 
                inplace = True 
            )

df_synthes["NBR SAC/PALETTE"] = df_synthes["Nombre Total SAC"] / df_synthes["Nombre De Palette Produite"]
df_synthes["Poid Tottal/PALETTE"] = df_synthes["Tottal Poids"] / df_synthes["Nombre De Palette Produite"]
df_synthes["OPT"] = round( df_synthes["Tottal Heure Travaile"] / dpt_golabl, 2 )
df_synthes["ECART PALETTE"] = df_synthes["OPT"] - df_synthes["Nombre De Palette Produite"]
df_synthes["OTS"] = round(df_synthes["OPT"]*64, 2)

df_synthes = df_synthes[["Shift", "Tottal Heure Travaile", "Nombre De Palette Produite", "OPT", "ECART PALETTE", 
                        "Nombre d'heure d'arret", "Cause", "Nombre Total SAC", "OTS", "Tottal Poids", 
                        "NBR SAC/PALETTE", "Poid Tottal/PALETTE", "Tottal Echantillon 10Kg", "Tottal Echantillon 4Kg"]]


In [16]:
[ 10 for i in range(df_synthes.shape[0])]

[10, 10, 10]

In [22]:
df_synthes

,Shift,Tottal Heure Travaile,Nombre De Palette Produite,OPT,ECART PALETTE,Nombre d'heure d'arret,Cause,Nombre Total SAC,OTS,Tottal Poids,NBR SAC/PALETTE,Poid Tottal/PALETTE,Tottal Echantillon 10Kg,Tottal Echantillon 4Kg
0,Shift_1,460.75,4,46.08,42.08,420.75,______Défaut mécanique mélangeur___Défaut élec...,36,2949.12,351.4,9.000000,87.850000,2,1
1,Shift_2,279.53,2,27.95,25.95,260.53,Manque de sable___,18,1788.80,175.7,9.000000,87.850000,1,1
2,Shift_3,415.50,3,41.55,38.55,385.50,___Défaut Bouche 2 Ensacheuse___,26,2659.20,250.9,8.666667,83.633333,1,2


In [76]:
df_synthes.to_excel(r"./Synthese.xlsx", index= False, startrow=2, startcol=2)

In [83]:
# Create a new Excel workbook
wb = Workbook()

# Select the active sheet (you can create a new one if needed)
sheet = wb.active

# Convert the DataFrame to a Pandas Excel writer object
writer = pd.ExcelWriter(r"./Synthese.xlsx", engine='openpyxl') 
writer.book = wb

# Write the DataFrame to the Excel sheet
df_synthes.to_excel(writer, sheet_name='Sheet1', index=False, startrow=1, startcol=1)

# Get the xlsxwriter workbook and worksheet objects
workbook = writer.book
worksheet = writer.sheets['Sheet1']

# Define a border style
border = Border(left=Side(border_style='thin'),
                right=Side(border_style='thin'),
                top=Side(border_style='thin'),
                bottom=Side(border_style='thin'))

# # Create fill colors for each column
# fill_colors = {
#     'Column1': PatternFill(start_color='FFFF00', end_color='FFFF00', fill_type='solid'),
#     'Column2': PatternFill(start_color='00FF00', end_color='00FF00', fill_type='solid'),
#     'Column3': PatternFill(start_color='FF00FF', end_color='FF00FF', fill_type='solid')
# }

# # Apply borders and colors to each column
# for col_idx, column_name in enumerate(df_synthes.columns, start=1):
#     # Apply the border to each cell in the column
#     for row_idx, cell in enumerate(sheet.iter_cols(min_col=col_idx, max_col=col_idx, min_row=2), start=1):
#         cell[0].border = border

#     # Apply the fill color to the column header cell
#     header_cell = sheet.cell(row=1, column=col_idx)
#     header_cell.fill = fill_colors.get(column_name, PatternFill(start_color='FFFFFF', end_color='FFFFFF', fill_type='solid'))


# Save the Excel file
writer.save()



C:\Users\zifellah\AppData\Local\Temp\ipykernel_18472\3676384356.py:9: FutureWarning: Setting the `book` attribute is not part of the public API, usage can give unexpected or corrupted results and will be removed in a future version
  writer.book = wb
C:\Users\zifellah\AppData\Local\Temp\ipykernel_18472\3676384356.py:43: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()


In [84]:
import pandas as pd

# Sample DataFrame
data = {'Column1': [1, 2, 3, 4],
        'Column2': ['A', 'B', 'C', 'D'],
        'Column3': [10, 20, 30, 40]}

df = pd.DataFrame(data)


In [85]:
excel_writer = pd.ExcelWriter('output.xlsx', engine='openpyxl')

In [86]:
df.to_excel(excel_writer, sheet_name='Sheet1', index=False)

In [87]:
workbook = excel_writer.book
worksheet = excel_writer.sheets['Sheet1']


In [88]:
from openpyxl.styles import Border, Side, PatternFill, Font

# Border formatting
border = Border(left=Side(border_style='thin'),
                right=Side(border_style='thin'),
                top=Side(border_style='thin'),
                bottom=Side(border_style='thin'))

# Fill (background color) formatting
colors = ['FFFF00', 'FF00FF', '00FFFF']  # Example colors
fills = [PatternFill(start_color=color, end_color=color, fill_type='solid') for color in colors]

# Font formatting (optional)
font = Font(bold=True, color='FFFFFF')

# Apply formatting to each column
for idx, column in enumerate(df.columns, 1):
    for cell in worksheet[f"{chr(65 + idx)}:"]:
        cell.border = border
        cell.fill = fills[idx - 1]
        cell.font = font


ValueError: B: is not a valid coordinate or range

In [133]:
from openpyxl.styles import Border, Side, PatternFill, Font
import pandas as pd
from openpyxl.utils import get_column_letter


def cemix_Synthese_to_excel(df, export):
    
    excel_writer = pd.ExcelWriter(export, engine='openpyxl')

    df.to_excel(excel_writer, sheet_name='Sheet1', index=False)
    cols = list(df.columns)

    workbook = excel_writer.book
    worksheet = excel_writer.sheets['Sheet1']

    for idx, column in enumerate(df.columns, 1):
        col_letter = get_column_letter(idx)
        worksheet.column_dimensions[col_letter].width = 17

    border = Border(left=Side(border_style='thin'),
                    right=Side(border_style='thin'),
                    top=Side(border_style='thin'),
                    bottom=Side(border_style='thin'))

    colors = ["FFFFFF" for i in df.columns] 
    for i in cols:
        if cols.index(i)%2:
            colors[cols.index(i)] = "CCCCCC"

    fills = [PatternFill(start_color=color, end_color=color, fill_type='solid') for color in colors]

    font = Font(bold=True, color='000000')

    for idx, column in enumerate(df.columns, 0):
        for row in worksheet.iter_rows(min_row=2, max_row=len(df) + 1, min_col=idx, max_col=idx):
            for cell in row:
                cell.border = border
                cell.fill = fills[idx - 1]
                cell.font = font

    # Save the Excel file
    excel_writer.save()
    print("The File is successfully exported in: ", export)


In [134]:
import pandas as pd
import numpy as np

# Set a random seed for reproducibility
np.random.seed(42)

# Create a DataFrame with 9 columns and 20 rows of random data
data = {
    'Column1': np.random.rand(20),
    'Column2': np.random.rand(20),
    'Column3': np.random.rand(20),
    'Column4': np.random.rand(20),
    'Column5': np.random.rand(20),
    'Column6': np.random.rand(20),
    'Column7': np.random.rand(20),
    'Column8': np.random.rand(20),
    'Column9': np.random.rand(20),
}

df = pd.DataFrame(data)

# Display the DataFrame
cemix_Synthese_to_excel(df, "./New_output.xlsx")


The File is successfully exported in:  ./New_output.xlsx


C:\Users\zifellah\AppData\Local\Temp\ipykernel_18472\723043056.py:42: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  excel_writer.save()


In [12]:
import os
def get_shifts(ligne):
    conn = sqlite3.connect("Cemix_database_test.db")
    query = f"""
                SELECT p.numero_palette, p.date, c.shift
                FROM palette p
                join cemix_info c on c.id = p.cemix_main_id
                WHERE 
                ((
                strftime('%H:%M:%S', 'now') >= '07:00:00' 
                AND p.date BETWEEN 
                                datetime('now', 'start of day', '-0 day', '07:00:00') AND 
                                datetime('now', 'start of day','+1 day', '07:00:00')
                )
                OR
                (
                strftime('%H:%M:%S', 'now') < '07:00:00' 
                AND p.date BETWEEN 
                                    datetime('now', 'start of day', '-1 day', '07:00:00') AND
                                    datetime('now', 'start of day', '+0 day', '07:00:00')
                ))
                and 
                c.ligne = '{ligne}' """
    
    df = pd.read_sql_query(query, conn)

    list_shifts = ["shift-1","shift-2","shift-3","shift-4","shift-5","shift-6"]
    list_shifts_used = list(set(df["shift"])) 

    shift_allowed = [item for item in list_shifts if item not in list_shifts_used]
    conn.close()

    shift_allowed_html = [{'label': shift.replace("s", "S").replace("-", " "), 'value': shift} for shift in shift_allowed]

    return shift_allowed_html

In [13]:
get_shifts("line_1")

[{'label': 'Shift 2', 'value': 'shift-2'},
 {'label': 'Shift 3', 'value': 'shift-3'},
 {'label': 'Shift 4', 'value': 'shift-4'},
 {'label': 'Shift 5', 'value': 'shift-5'},
 {'label': 'Shift 6', 'value': 'shift-6'}]